In [1]:
# Cell 1 — Install deps (no XRootD needed)
!pip -q install "uproot>=5" awkward vector rich tqdm pandas pyarrow fastparquet matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.8/393.8 kB 10.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.7/656.7 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.3 MB/s eta 0:00:00


In [2]:
import uproot
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt


In [3]:
FEATURES = [
"nMuon","nElectron","nJet","MET_pt","MET_phi","MET_sumEt",
"Muon_pt_0","Muon_eta_0","Muon_phi_0",
"Muon_pt_1","Muon_eta_1","Muon_phi_1",
"Electron_pt_0","Electron_eta_0","Electron_phi_0",
"Electron_pt_1","Electron_eta_1","Electron_phi_1",
"Jet_pt_0","Jet_eta_0","Jet_phi_0",
"Jet_pt_1","Jet_eta_1","Jet_phi_1",
"Jet_pt_2","Jet_eta_2","Jet_phi_2",
"Jet_pt_3","Jet_eta_3","Jet_phi_3",
"HT","ST","M_ll","M_jj_01","M_jj_12",
"delta_phi_MET_j0","delta_phi_MET_j1","min_delta_phi_MET_jets",
"delta_R_j0_j1","delta_phi_ll","delta_R_ll",
"Jet_btagDeepB_0","Jet_btagDeepB_1",
"MT_lep_MET","HT_ratio","MET_pt_HT_ratio",
"nJet_pt30","Jet_mass_0","LeadLepton_pt","sum_pt_leptons"
]


In [4]:
import uproot
import pandas as pd
import glob

def find_tree_recursive(file):
    """Find first TTree anywhere inside ROOT file."""
    for key, obj in file.items(recursive=True):
        if isinstance(obj, uproot.behaviors.TTree.TTree):
            return obj
    return None


def load_root_folder_chunked(
        folder,
        label,
        source,
        step_size=50000,
        max_files=None,
        max_events=None):

    files = sorted(glob.glob(folder + "/*.root"))
    if max_files:
        files = files[:max_files]

    dfs = []
    total_events = 0

    print(f"\nLoading {source} from {len(files)} files")

    for i, f in enumerate(files):

        print(f"  File {i+1}/{len(files)}")

        with uproot.open(f) as file:

            tree = find_tree_recursive(file)

            if tree is None:
                print("   No tree found, skipping")
                continue

            print("   Found tree:", tree.name)

            for batch in tree.iterate(
                    FEATURES,
                    library="pd",
                    step_size=step_size):

                batch["label"] = label
                batch["source"] = source
                dfs.append(batch)

                total_events += len(batch)

                if max_events and total_events >= max_events:
                    print(f"Reached event limit: {max_events}")
                    return pd.concat(dfs, ignore_index=True).iloc[:max_events]

    if len(dfs) == 0:
        print("No data loaded!")
        return pd.DataFrame()

    return pd.concat(dfs, ignore_index=True)


In [5]:
cmsderived = "/kaggle/input/datasets/katakuricharlotte/cms-derivedroot/derivedroot"
cms2016G   = "/kaggle/input/datasets/hiteshrs/cms2016g29-5785/processed_events"

dy   = load_root_folder_chunked(f"{cmsderived}/DYJetsToLL_0J_TuneCP5", 0, "DY", max_files=2)
wj   = load_root_folder_chunked(f"{cmsderived}/WJetsToLNu_TuneCP5", 0, "WJets", max_files=2)
tt   = load_root_folder_chunked(f"{cmsderived}/TTJets_TuneCP5", 0, "TT", max_files=2)
susy = load_root_folder_chunked(f"{cmsderived}/SMS-TChiWZ_ZToLL", 1, "SUSY", max_files=2)
real = load_root_folder_chunked(cms2016G, -1, "REAL", max_files=2)



Loading DY from 2 files
  File 1/2
   Found tree: Events
  File 2/2
   Found tree: Events

Loading WJets from 2 files
  File 1/2
   Found tree: Events
  File 2/2
   Found tree: Events

Loading TT from 2 files
  File 1/2
   Found tree: Events
  File 2/2
   Found tree: Events

Loading SUSY from 2 files
  File 1/2
   Found tree: Events
  File 2/2
   Found tree: Events

Loading REAL from 2 files
  File 1/2
   No tree found, skipping
  File 2/2
   No tree found, skipping
No data loaded!


In [6]:
df = pd.concat([dy, wj, tt, susy, real], ignore_index=True)

df_clean = df.copy()

print("Initial events:", len(df_clean))


Initial events: 11615070


In [7]:
import numpy as np

df_clean.replace([np.inf, -np.inf], np.nan, inplace=True)
df_clean.dropna(inplace=True)

print("After NaN removal:", len(df_clean))


After NaN removal: 11615070


In [8]:
# transverse momenta upper bounds (detector realism)
df_clean = df_clean[
    (df_clean["MET_pt"] < 2000) &
    (df_clean["Jet_pt_0"] < 3000) &
    (df_clean["Muon_pt_0"] < 2000) &
    (df_clean["Electron_pt_0"] < 2000)
]

# eta detector acceptance
eta_cols = [c for c in df_clean.columns if "_eta_" in c]

for c in eta_cols:
    df_clean = df_clean[df_clean[c].abs() < 5]

print("After physics cleaning:", len(df_clean))


After physics cleaning: 11538158


In [9]:
df_clean = df_clean[
    (df_clean["nJet"] > 0) |
    (df_clean["nMuon"] > 0) |
    (df_clean["nElectron"] > 0)
]

print("After trivial-event removal:", len(df_clean))


After trivial-event removal: 11103638


In [10]:
baseline = df_clean[
    (df_clean["MET_pt"] > 50) &
    (df_clean["nJet"] >= 2) &
    (df_clean["HT"] > 200) &
    (df_clean["LeadLepton_pt"] > 20)
]

print("Baseline events:", len(baseline))


Baseline events: 659920


In [11]:
signal_region = baseline[
    (baseline["MET_pt"] > 200) &
    (baseline["HT"] > 400) &
    (baseline["nJet"] >= 3)
]

print("Signal region events:", len(signal_region))


Signal region events: 60638


In [12]:
control_region = baseline[
    (baseline["MET_pt"] < 100) &
    (baseline["HT"] < 300)
]

print("Control region events:", len(control_region))


Control region events: 118988


In [13]:
from sklearn.model_selection import train_test_split

train_df = baseline[baseline["label"] != -1]

X = train_df.drop(columns=["label","source"])
y = train_df["label"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("Train:", len(X_train))
print("Validation:", len(X_val))


Train: 527936
Validation: 131984


In [14]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# log-transform heavy-tailed variables
log_cols = ["MET_pt","HT","ST","LeadLepton_pt","Jet_pt_0"]

for c in log_cols:
    X_train[c] = np.log1p(X_train[c])
    X_val[c]   = np.log1p(X_val[c])

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val   = scaler.transform(X_val)


In [15]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method="hist",
    eval_metric="logloss"
)

model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, ...)

In [16]:
from sklearn.metrics import roc_auc_score

pred = model.predict_proba(X_val)[:,1]
auc = roc_auc_score(y_val, pred)

print("Validation AUC:", auc)


Validation AUC: 0.9741243127630431


In [17]:
print("Total real events:", (df_clean["label"]==-1).sum())
print("Real after baseline:", (baseline["label"]==-1).sum())
print("Real in signal region:", (signal_region["label"]==-1).sum())


Total real events: 0
Real after baseline: 0
Real in signal region: 0


In [18]:
real_df = baseline[baseline["label"] == -1]

X_real = real_df.drop(columns=["label","source"])

for c in log_cols:
    X_real[c] = np.log1p(X_real[c])

X_real = scaler.transform(X_real)

real_scores = model.predict_proba(X_real)[:,1]

real_df["susy_score"] = real_scores


ValueError: Found array with 0 sample(s) (shape=(0, 50)) while a minimum of 1 is required by StandardScaler.

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), dpi=120)

ax.hist(real_scores, bins=100, histtype="step", linewidth=1.8)

ax.set_xlabel("SUSY probability")
ax.set_ylabel("Events")

paper_axes(ax)
fig.tight_layout()

fig.savefig("/kaggle/working/real_susy_scores.jpg", dpi=300)
plt.show()


In [ ]:
real_df.sort_values("susy_score", ascending=False).head(20)
